Assignment 3 : Colin Kirby

In [2]:
# Imports.
import pygame
import moderngl
import numpy as np

pygame 2.6.0 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# Generate T Values from -1 to 1.
t_vals = np.linspace(-1, 1, 3000)

# Initialize Empty List.
x_vals = []
y_vals = []

# Loop Through Values Setting X and Y for Heart Shape.
for t in t_vals:
    x = np.sin(t) * np.cos(t) * np.log(np.abs(t))
    y = (np.abs(t) ** 0.3) * (np.sqrt(np.cos(t))) - 0.5

    # Append Calculated Values to List.
    x_vals.append(x)
    y_vals.append(y)

# Set X Values and Y Values to Numpy Array.
x_vals = np.array(x_vals)
y_vals = np.array(y_vals)

# Combine x_vals and y_vals into a vertices array along with 0 for z and 1 for w.
vertices = np.column_stack((x_vals, y_vals, np.zeros_like(x_vals), np.ones_like(x_vals))).astype(np.float32)

In [4]:
# Initialize Pygame and Set up window.
pygame.init
pygame.display.set_mode((800, 600), pygame.OPENGL | pygame.DOUBLEBUF)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MAJOR_VERSION,4)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MINOR_VERSION,3)
pygame.display.set_caption(title = "Assignment 3 : Colin Kirby")

In [5]:
# Create OpenGL context.
gl = moderngl.get_context()

In [6]:
# Create GPU Buffer for Vertex Data.
vertex_buf = gl.buffer(vertices)

In [7]:
vertex_shader_code = '''
#version 330 core

layout (location = 0) in vec4 in_vertex;
uniform float zoomFactor;
uniform float aspect;

void main (){
    gl_Position = vec4(in_vertex.x/(zoomFactor*aspect), in_vertex.y/(zoomFactor), in_vertex.zw);
}
'''

In [8]:
fragment_shader_code = '''
#version 330 core

layout (location = 0) out vec4 out_color;

void main() {
    out_color = vec4(1, 0, 1, 1);
}
'''


In [9]:
# Create new shader program in OpenGl.
program = gl.program(vertex_shader=vertex_shader_code, fragment_shader=fragment_shader_code)

In [10]:
# Create Vertex Array Obj.
renderable = gl.vertex_array(program, [(vertex_buf, '4f', 'in_vertex')]) 

In [11]:
# Clear Background Screen.
gl.clear(0.2,0.2,0.2)

# Render the VAO using Shader Program.
renderable.render()

# Update display with Rendered Content.
pygame.display.flip()

In [ ]:
# Variables for Zoom and Aspect Ratio.
zoomFactor = 1.0
deltaZoom = 0.025
min_zoom = 1/5
max_zoom = 5
zoom_in = True
aspect = 800 / 600 

# Rendering Loop.
running = True
clock = pygame.time.Clock()

# While Running.
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
            running = False

    # Update Zoom Factor.
    if zoom_in:
        zoomFactor += deltaZoom
        if zoomFactor >= max_zoom:
            zoom_in = False
    else:
        zoomFactor -= deltaZoom
        if zoomFactor <= min_zoom:
            zoom_in = True

    # Set Zoom and Aspect Variables.
    program['zoomFactor'].value = zoomFactor
    program['aspect'].value = aspect

    # Clear Screen and Render.
    gl.clear(0.2, 0.2, 0.2)  # Background color
    renderable.render(moderngl.TRIANGLE_FAN)

    # Swap Buffer.
    pygame.display.flip()
    clock.tick(60)  # 60 FPS

# Quit Pygame.
pygame.quit()
